Recursos:
 - https://docs.datastax.com/en/developer/python-driver/3.25/getting_started/
 - https://cassandra.apache.org/doc/latest/cassandra/faq/
 - https://openwebinars.net/blog/como-usar-apache-cassandra-con-python/
 - https://docs.datastax.com/en/cql-oss/3.3/cql/cql_reference/cqlCreateTable.html
 - https://www.datastax.com/blog/most-important-thing-know-cassandra-data-modeling-primary-key

## Testeando la conexión a cassandra y algunas queries

In [62]:
import uuid

In [1]:
from cassandra.cluster import Cluster

#Conexion al cluster
#Es importante haber mapeado el puerto usando -p 9042:9042 al crear el contenedor

cluster = Cluster(['127.0.0.1'], port=9042)
#session = cluster.connect('bdnosql') #El keyspace lo creé desde la terminal

In [8]:
session = cluster.connect()

session.execute(
    """
    CREATE KEYSPACE bdnosql 
    WITH replication = {'class' : 'SimpleStrategy', 'replication_factor':1};
    """
)
session.set_keyspace('bdnosql')

session.execute(
    """
    CREATE TABLE estaciones(
    id int PRIMARY KEY,
    nombre text
    );
    """
)

In [10]:
# Insertar cosas a la tabla estaciones 
sentencia = session.prepare( 
    """
    INSERT INTO estaciones (id, nombre)
    VALUES (?, ?)
    """
)

row = (2, 'Verano')
session.execute(sentencia, row)

In [11]:
row = (3, 'Otoño')
resultado = session.execute(sentencia, row)

print(resultado)

In [15]:
# Leer cosas de la tabla
rows = session.execute('SELECT id, nombre FROM estaciones')
for row in rows:
    print(row.id, row.nombre)

5 OtraEstacion
2 Verano
4 Invierno
3 Otoño


In [13]:
sentencia_2 = session.prepare("SELECT id, nombre FROM estaciones WHERE id=?")
rows = session.execute(sentencia_2, [3])
for row in rows:
    print(row[0], row[1])

3 Otoño


In [14]:
#Insertando desde un dataFrame de pandas
import pandas as pd

data = [[4, 'Invierno'], [5, 'OtraEstacion']]
data = pd.DataFrame(data)

for row in data.values:
    session.execute(sentencia, row)

## Reconstrucción de tablas para precargarlas a base de datos

Para cargar los datos queremos:
 - Definir las tablas dentro del keyspace
 - Definir los métodos para popularlas
 - Cargar los datos 

 Dudas:
- ¿Las categorías están preestablecidas o cada usuario puede crear nuevas?

Consideraciones:
- El usuario al ingresar un nuevo libro no conoce el id, solo el título. Debe ser posible acceder al id del libro a partir de su nombre.

In [29]:
data = pd.read_csv('DatosBBDDE_test.csv')
data.columns = ['categoria', 'usuario', 'libro', 'calificacion']
data = data[['usuario', 'libro', 'categoria', 'calificacion']]

In [33]:
data.head()

,usuario,libro,categoria,calificacion
0,Yeudiel,Harry Potter,Fantasia,5
1,Eduardo,One Shot,Suspenso,5
2,FernandoA,On the road,Novela,5
3,Edgar,The Pillars of the Earth,Novela,4
4,Daniel,100 años de soledad,Realismo mágico,5


## Creación de keyspace y tablas

In [7]:
# Creación del keyspace
session.execute(
    """
    CREATE KEYSPACE bd_libros 
    WITH replication = {'class' : 'SimpleStrategy', 'replication_factor':1};
    """
)
session.set_keyspace('bd_libros')


### Creación de las tablas
#1. USUARIOS
session.execute(
    """
    CREATE TABLE clientes (
    id_cliente uuid PRIMARY KEY,
    nombre_cliente text,
    pais text,
    tipo_membresia int
    );
    """
)

#2. LIBROS
session.execute(
    """
    CREATE TABLE libros (
    id_libro uuid,
    titulo_libro text PRIMARY KEY,
    );
    """
)

#3. LIBROS POR CLIENTE
session.execute(
    """
    CREATE TABLE libros_por_cliente (
    id_cliente uuid,
    id_libro uuid,
    calificacion int,
    categoria text,
    PRIMARY KEY(id_cliente, id_libro, categoria)
    );
    """
)

#4. CATEGORIAS POR CLIENTE (maybe no es necesaria y con la 3 es suficiente)
session.execute(
    """
    CREATE TABLE categorias_por_cliente (
    id_cliente uuid,
    calificacion_promedio int,
    categoria text,
    PRIMARY KEY(id_cliente, calificacion_promedio, categoria))
    WITH CLUSTERING ORDER BY (calificacion_promedio DESC)
    ;
    """
)

#5. CLIENTES POR LIBRO
session.execute(
    """
    CREATE TABLE clientes_por_libro (
    id_libro uuid,
    calificacion int,
    id_cliente uuid,
    PRIMARY KEY(id_libro, calificacion, id_cliente))
    WITH CLUSTERING ORDER BY (calificacion DESC)
    ;
    """
)

#6. LIBROS POR CATEGORIA
session.execute(
    """
    CREATE TABLE libros_por_categoria (
    categoria text,
    id_libro uuid,
    id_cliente uuid,
    calificacion int,
    PRIMARY KEY (categoria, id_libro, id_cliente),
    );
    """
)

Quizá no es necesario tener un id_libro y es suficiente con usar el nombre del libro como identificador único.

## Queries

In [ ]:
def Q2_agregar_calificacion(id_cliente:str , titulo_libro:str, categoria:str, calificacion:int):
    q_obtener_id_libro = session.prepare(
        """
        SELECT id_libro FROM libros 
        WHERE titulo_libro=?
        """
    )
    q_agregar_libro = session.prepare( 
        """
        INSERT INTO libros (id_libro, titulo_libro)
        VALUES (?, ?)
        """
    )
    q_insertar_libros_por_cliente = session.prepare(
        """
        INSERT INTO libros_por_cliente (id_cliente, id_libro, calificacion, categoria)
        VALUES (?, ?, ?, ?)
        """
    )
    q_insertar_categorias_por_cliente = session.prepare(
        """
        """
    )
    q_insertar_clientes_por_libro = session.prepare(
        """
        INSERT INTO clientes_por_libro (id_libro, calificacion, id_cliente)
        VALUES (?, ?, ?)
        """
    )
    q_insertar_libros_por_categoria = session.prepare(
        """
        INSERT INTO clientes_por_libro (categoria, id_libro, calificacion, id_cliente)
        VALUES (?, ?, ?, ?)
        """
    )

    row = session.execute(q_obtener_id_libro, [titulo_libro])

    if row.one():
        id_libro = row.one().id_libro
    else:
        id_libro = uuid.uuid1()
        session.execute(q_agregar_libro, (id_libro, titulo_libro))

    session.execute(q_insertar_libros_por_cliente, (id_cliente, id_libro, calificacion, categoria))
    session.execute(q_insertar_clientes_por_libro, (id_libro, calificacion, id_cliente))
    session.execute(q_insertar_libros_por_categoria, (categoria, id_libro, calificacion, id_cliente))

    return     

In [ ]:
def Q3_categoría_preferida_por_cliente(id_cliente:str):
    
    resultado_cat_mayor_calif_promedio = session.execute(
        """
        SELECT categoria, AVG(calificacion) FROM libros_por_cliente
        WHERE id_cliente=?
        GROUP BY categoria
        ORDER BY AVG(calificacion)
        LIMIT 1
        """,
        [id_cliente]
    )
    
    return resultado_cat_mayor_calif_promedio.one()

In [ ]:
def Q4_mas_disfrutaron_libro(id_libro:str):
    
    resultado_max_calif = session.execute(
        """
        SELECT MAX(calificacion) FROM clientes_por_libro
        WHERE id_libro=%s
        LIMIT 1
        """,
        [id_libro]
    )

    if resultado_max_calif is None:
        return None

    resultado_clientes = session.execute(
        """
        SELECT id_cliente FROM clientes_por_libro
        WHERE id_libro=%s
        AND calificacion=%s
        """, 
        (id_libro, resultado_max_calif.one()[0])
    )
    
    return resultado_clientes.all()


In [ ]:
def Q5_mejores_libros_por_categoria(categoria:str, n:int=3):
    
    res = session.execute(
        """
        SELECT id_libro, AVG(calificacion) FROM libros_por_categoria
        WHERE categoria=%s
        GROUP BY id_libro
        ORDER BY AVG(calificacion)
        LIMIT %s
        """,
        (categoria, n)
    )
    return res.all()